<a href="https://colab.research.google.com/github/mickeyrahm/Portfolio/blob/master/notebooks/starter_publishing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
restart = True
epoch_to_pickup = 0

In [2]:
# Import libraries

from tensorflow.keras.layers import StringLookup
import numpy as np
import os
import time
import random
import contextlib
import io
import re
import string
import gc  # Import the garbage collector module

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers import TextVectorization

In [3]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [4]:
path = ''

In [5]:
# from google.colab import drive
# drive.mount('/content/drive')
# path = '/content/drive/My Drive/M6_Fall2023e/'

## Functions for downloading text


In [6]:
def preprocess_text(text):

    # Remove Project Gutenberg boilerplate
    text = text.replace("Project Gutenberg", "")
    text = text.replace("Gutenberg", "")

    # Optionally trim header/footer:
    text = text[500:-5000]

    # Remove carriage returns
    text = text.replace("\r", "")

    # Fix quotes
    text = text.replace("“", "\"")
    text = text.replace("”", "\"")

    # Remove strange symbols
    text = re.sub(r'[_*—]', '', text)

    # Normalize whitespace
    text = re.sub(r'\s+', ' ', text)

    # Lowercase everything first (your method handles capitals with ^ symbols, so skip this if using that)
    # text = text.lower()

    # Split into sentences if you want (optional)
    # import nltk
    # nltk.download('punkt')
    # from nltk.tokenize import sent_tokenize
    # sentences = sent_tokenize(text)
    # sentences = [s for s in sentences if len(s) > 10]
    # text = ' '.join(sentences)

    # Remove duplicate whitespace again after sentence processing
    text = re.sub(r'\s+', ' ', text)

    # Replace any capital letter at the start of a word with ^ followed by the lowercase letter
    text = re.sub(r"(?<![a-zA-Z])([A-Z])", lambda match: f"^{match.group(0).lower()}", text)

    # Replace all other capital letters with lowercase
    text = re.sub(r"([A-Z])", lambda match: f"{match.group(0).lower()}", text)

    # Remove duplicate whitespace
    text = re.sub(r"\s+", " ", text)
    text = re.sub(r"\n+", "\n", text)
    text = re.sub(r"\t+", "\t", text)

    # Replace whitespace characters with special words
    text = re.sub(r"(\t)", r" zztabzz ", text)
    text = re.sub(r"(\n)", r" zznewlinezz ", text)
    text = re.sub(r"(\s)", r" zzspacezz ", text)

    # Split before and after punctuation
    for punctuation in string.punctuation:
        text = text.replace(punctuation, f" {punctuation} ")

    return text


In [7]:
def postprocess_text(text):

    # Replace special words with whitespace characters
    text = text.replace("zztabzz", "\t")
    text = text.replace("zznewlinezz", "\n")
    text = text.replace("zzspacezz", " ")

    # Remake capital letters at beginning of words
    text = re.sub(r"\^([a-z])", lambda match: f"{match.group(1).upper()}", text)

    text = text.replace("^", "")

    return text

In [8]:
import os
import tensorflow as tf

def getMyText():
    file_name = 'gatsby.txt'
    file_url = 'https://www.gutenberg.org/files/64317/64317-0.txt'
    local_dir = 'saved_files'  # Directory to save the file
    local_path = os.path.join(local_dir, file_name)

    try:
        # Ensure the directory exists
        if not os.path.exists(local_dir):
            os.makedirs(local_dir)

        # Check if the file exists locally
        if os.path.exists(local_path):
            print(f"File '{file_name}' found locally. Using it.")
        else:
            print(f"File '{file_name}' not found locally. Downloading it.")
            # Download the file
            downloaded_path = tf.keras.utils.get_file(file_name, file_url)

            # Save the downloaded file to the designated local directory
            with open(downloaded_path, 'rb') as source_file:
                with open(local_path, 'wb') as dest_file:
                    dest_file.write(source_file.read())

        # Read the file's contents
        with open(local_path, 'rb') as file:
            text = file.read().decode(encoding='utf-8')

        return preprocess_text(text)

    except Exception as e:
        print(f"An error occurred: {e}")
        return None


In [9]:
getMyText()

File 'gatsby.txt' found locally. Using it.


'ld - hatted ,  zzspacezz high - bouncing zzspacezz lover ,  zzspacezz  ^ i zzspacezz must zzspacezz have zzspacezz you !  "  zzspacezz  ^ thomas zzspacezz  ^ parke zzspacezz d’ ^ invilliers zzspacezz  ^ i zzspacezz  ^ in zzspacezz my zzspacezz younger zzspacezz and zzspacezz more zzspacezz vulnerable zzspacezz years zzspacezz my zzspacezz father zzspacezz gave zzspacezz me zzspacezz some zzspacezz advice zzspacezz that zzspacezz  ^ i’ve zzspacezz been zzspacezz turning zzspacezz over zzspacezz in zzspacezz my zzspacezz mind zzspacezz ever zzspacezz since .  zzspacezz  "  ^ whenever zzspacezz you zzspacezz feel zzspacezz like zzspacezz criticizing zzspacezz anyone ,  "  zzspacezz he zzspacezz told zzspacezz me ,  zzspacezz  " just zzspacezz remember zzspacezz that zzspacezz all zzspacezz the zzspacezz people zzspacezz in zzspacezz this zzspacezz world zzspacezz haven’t zzspacezz had zzspacezz the zzspacezz advantages zzspacezz that zzspacezz you’ve zzspacezz had .  "  zzspacezz  ^ he z

In [10]:
def getRandomText(numbooks = 1, verbose=False):
  download_log = io.StringIO()
  text_random = ''
  for b in range(numbooks):
    foundbook = False
    while(foundbook == False):
      booknum = random.randint(100,60000)
      if verbose:
        print('Trying Book #: ',booknum)
      if random.random() > 0.5:
        url = 'https://www.gutenberg.org/files/' + str(booknum) + '/' + str(booknum) + '-0.txt'
        filename_temp = str(booknum) + '-0.txt'
      else:
        url = 'https://www.gutenberg.org/cache/epub/' + str(booknum) + '/pg' + str(booknum) + '.txt'
        filename_temp = 'pg' + str(booknum) + '.txt'
      if verbose:
        print('Trying: ', url)
      try:
        if verbose:
          path_to_file_temp = tf.keras.utils.get_file(filename_temp, url)
        else:
          with contextlib.redirect_stdout(download_log):
            path_to_file_temp = tf.keras.utils.get_file(filename_temp, url)
        temptext = open(path_to_file_temp, 'rb').read().decode(encoding='utf-8')
        tf.io.gfile.remove(path_to_file_temp)
        if (temptext.find('Language: English') >= 0):
          offset = random.randint(-20,20)
          header = 2000
          total_length = 200000
          chopoffend = 10000
          if len(temptext) > (header+total_length+offset+chopoffend):
            foundbook = True
            text_random += temptext[header+offset:header+total_length+offset]
            #print("Yes: " + str(booknum))
            if verbose:
              print('New size of dataset: ', len(text_random))
          elif len(temptext) > (header+12000):
            foundbook = True
            text_random += temptext[header:-chopoffend]
            #print("Yes (smaller): " + str(booknum))
            if verbose:
              print('New size of dataset: ', len(text_random))
          else:
            if verbose:
              print('Not long enough. Trying again...')
            #print("No: " + str(booknum) + " too short")
        else:
          if verbose:
            print('Not English. Trying again...')
          #print("No: " + str(booknum) + " not English")
        del temptext
      except:
        if verbose:
          print('Not valid file. Trying again...')
        #print("No: " + str(booknum) + " not valid")
        foundbook = False
    if verbose:
      print("Found " + str(b+1) + " books so far...")
  del download_log
  #text_random = "".join(c for c in text_random if c in vocab)
  #all_ids_random = ids_from_chars(tf.strings.unicode_split(text_random, 'UTF-8'))
  #ids_dataset_random = tf.data.Dataset.from_tensor_slices(all_ids_random)
  #sequences_random = ids_dataset_random.batch(seq_length+1, drop_remainder=True)
  #dataset_random = sequences_random.map(split_input_target)
  #dataset_random = (dataset_random.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE))
  #return dataset_random
  return preprocess_text(text_random)

In [11]:
if restart:
  vocab_text = getMyText()

File 'gatsby.txt' found locally. Using it.


Make vocabulary (Adapted from TensorFlow word embedding tutorial)

---



In [12]:
# Vocabulary size and number of words in a sequence.
vocab_size = 6191
sequence_length = 128

In [13]:
if restart:
  # Use the text vectorization layer to normalize, split, and map strings to
  # integers. Note that the layer uses the custom standardization defined above.
  # Set maximum_sequence length as all samples are not of the same length.
  vectorize_layer = TextVectorization(
      standardize='lower',
      split='whitespace',
      max_tokens=vocab_size,
      output_mode='int',
      #output_sequence_length=sequence_length
      )

In [14]:
if restart:
  # Make a text-only dataset (no labels) and call adapt to build the vocabulary.
  vectorize_layer.adapt([vocab_text])

In [15]:
if restart:
  vocabulary = vectorize_layer.get_vocabulary()

Save Vocabulary

In [16]:
if restart:
  with open(path + "vocabulary.txt", "w") as file:
    for word in vocabulary:
        file.write(word + "\n")

Load Saved Vocabulary

In [17]:
if restart == False:
  with open(path + "vocabulary.txt", "r") as file:
      vocabulary = [word.strip() for word in file.readlines()]
      vocabulary = vocabulary

  vectorize_layer = TextVectorization(
      vocabulary=vocabulary,
      standardize='lower',
      split='whitespace',
      max_tokens=vocab_size,
      output_mode='int',
      #output_sequence_length=sequence_length
      )

In [18]:
print(vocabulary[:20])
print(vocabulary[-20:])

['', '[UNK]', np.str_('zzspacezz'), np.str_('^'), np.str_('.'), np.str_(','), np.str_('"'), np.str_('the'), np.str_('-'), np.str_('and'), np.str_('a'), np.str_('i'), np.str_('to'), np.str_('of'), np.str_('in'), np.str_('he'), np.str_('was'), np.str_('that'), np.str_('it'), np.str_('his')]
[np.str_('absent'), np.str_('abrupt'), np.str_('abroad'), np.str_('abrams'), np.str_('aboutthings'), np.str_('abounding'), np.str_('abounded'), np.str_('abnormal'), np.str_('abandon'), np.str_('aalmost'), np.str_('9'), np.str_('5th'), np.str_('33rd'), np.str_('3'), np.str_('1922'), np.str_('1915'), np.str_('1906'), np.str_('1902'), np.str_('158th'), np.str_('12')]


Turn text into a dataset

In [19]:
# This function will generate our sequence pairs:
def split_input_target(sequence):
    input_ids = sequence[:-1]
    target_ids = sequence[1:]
    return input_ids, target_ids

# This function will create the dataset
def text_to_dataset(text):
  all_ids = vectorize_layer(text)
  ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)
  del all_ids
  sequences = ids_dataset.batch(sequence_length+1, drop_remainder=True)
  del ids_dataset

  # Call the function for every sequence in our list to create a new dataset
  # of input->target pairs
  dataset = sequences.map(split_input_target)
  del sequences

  # shuffle


  return dataset

Test on vocab text

In [20]:
if restart:
  vocab_ds = text_to_dataset(vocab_text)

In [21]:
def text_from_ids(ids):
  text = ''.join([vocabulary[index] for index in ids])
  return postprocess_text(text)

vocabulary_adjusted = vocabulary
vocabulary_adjusted[0] = '[UNK]'
vocabulary_adjusted[1] = ''

words_from_ids = tf.keras.layers.StringLookup(vocabulary=vocabulary_adjusted, invert=True)

In [22]:
if restart:
  for input_example, target_example in vocab_ds.take(1):
    print("Input: ")
    print(input_example)
    print(text_from_ids(input_example))
    print(words_from_ids(input_example))
    print("Target: ")
    print(target_example)
    print(text_from_ids(target_example))

Input: 
tf.Tensor(
[4427    8 4740    5    2  375    8 5840    2 4368    5    2    3   11
    2  222    2   54    2   20   57    6    2    3 1922    2    3 4045
    2 5213    3 4535    2    3   11    2    3   14    2   33    2 1832
    2    9    2   95    2 2908    2  189    2   33    2  726    2  254
    2   28    2   70    2 2733    2   17    2    3  178    2   65    2
  500    2   53    2   14    2   33    2  546    2  239    2  506    4
    2    6    3  752    2   20    2  606    2   61    2 5468    2  615
    5    6    2   15    2  165    2   28    5    2    6   72    2  347
    2   17    2   36    2    7    2  132    2   14    2   45    2  250
    2  452], shape=(128,), dtype=int64)
ld-hatted, high-bouncing lover, I must have you!" Thomas Parke d’Invilliers I In my younger and more vulnerable years my father gave me some advice that I’ve been turning over in my mind ever since. "Whenever you feel like criticizing anyone," he told me, "just remember that all the people in this wor

In [23]:
BATCH_SIZE = 128
BUFFER_SIZE = 10000

def setup_dataset(dataset):
  dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))
  return dataset


In [24]:
if restart:
  vocab_ds = setup_dataset(vocab_ds)

## III. Build the model

Next, we'll build our model. Up until this point, you've been using the Keras symbolic, or imperative API for creating your models. Doing something like:

    model = tf.keras.models.Sequentla()
    model.add(tf.keras.layers.Dense(80, activation='relu))
    etc...

However, tensorflow has another way to build models called the Functional API, which gives us a lot more control over what happens inside the model. You can read more about [the differences and when to use each here](https://blog.tensorflow.org/2019/01/what-are-symbolic-and-imperative-apis.html).

We'll use the functional API for our RNN in this example. This will involve defining our model as a custom subclass of `tf.keras.Model`.

If you're not familiar with classes in python, you might want to review [this quick tutorial](https://www.w3schools.com/python/python_classes.asp), as well as [this one on class inheritance](https://www.w3schools.com/python/python_inheritance.asp).

Using a functional model is important for our situation because we're not just training it to predict a single character for a single sequence, but as we make predictions with it, we need it to remember those predictions as use that memory as it makes new predictions.


In [25]:
from tensorflow.keras import layers, Model

class GatsbyTextModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__()

        self.embedding = layers.Embedding(vocab_size, embedding_dim)

        # Bidirectional LSTM #1
        self.lstm1 = layers.Bidirectional(
            layers.LSTM(
                rnn_units,
                return_sequences=True,
                return_state=True,
                recurrent_dropout=0.2,
            )
        )
        self.norm1 = layers.LayerNormalization()

        # Bidirectional LSTM #2
        self.lstm2 = layers.Bidirectional(
            layers.LSTM(
                rnn_units,
                return_sequences=True,
                return_state=True,
                recurrent_dropout=0.2,
            )
        )
        self.norm2 = layers.LayerNormalization()

        self.dense = layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
      x = self.embedding(inputs, training=training)

      batch_size = tf.shape(inputs)[0]

      if states is not None:
          # Reshape previous states to match dynamic batch size
          state1 = [tf.reshape(s, [batch_size, -1]) for s in states[0]]
          state2 = [tf.reshape(s, [batch_size, -1]) for s in states[1]]
      else:
          state1 = [
              tf.zeros([batch_size, self.lstm1.forward_layer.units]),
              tf.zeros([batch_size, self.lstm1.forward_layer.units]),
              tf.zeros([batch_size, self.lstm1.forward_layer.units]),
              tf.zeros([batch_size, self.lstm1.forward_layer.units]),
          ]
          state2 = [
              tf.zeros([batch_size, self.lstm2.forward_layer.units]),
              tf.zeros([batch_size, self.lstm2.forward_layer.units]),
              tf.zeros([batch_size, self.lstm2.forward_layer.units]),
              tf.zeros([batch_size, self.lstm2.forward_layer.units]),
          ]

      x, forward_h1, forward_c1, backward_h1, backward_c1 = self.lstm1(
          x, initial_state=state1, training=training
      )
      x = self.norm1(x)

      x, forward_h2, forward_c2, backward_h2, backward_c2 = self.lstm2(
          x, initial_state=state2, training=training
      )
      x = self.norm2(x)

      x = self.dense(x, training=training)

      if return_state:
          states_out = [
              [forward_h1, forward_c1, backward_h1, backward_c1],
              [forward_h2, forward_c2, backward_h2, backward_c2]
          ]
          return x, states_out
      else:
          return x



In [26]:
if restart:
  dataset = vocab_ds
  del vocab_text
  del vocab_ds
else:
  new_text = getRandomText(numbooks = 10)
  dataset = text_to_dataset(new_text)
  del new_text
  dataset = setup_dataset(dataset)

In [27]:
# Create an instance of our model
#vocab_size=len(ids_from_chars.get_vocabulary())
embedding_dim = 256
rnn_units = 512

model = GatsbyTextModel(vocab_size, embedding_dim, rnn_units)

In [28]:
# Verify the output of our model is correct by running one sample through
# This will also compile the model for us. This step will take a bit.
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")


(128, 128, 6191) # (batch_size, sequence_length, vocab_size)


In [29]:
# Now let's view the model summary
model.summary()

Model: "gatsby_text_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (128, 128, 256)        │     1,584,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ((128, 128, 1024),     │     3,149,824 │
│                                 │ (128, 512), (128,      │               │
│                                 │ 512), (128, 512),      │               │
│                                 │ (128, 512))            │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization             │ (128, 128, 1024)       │         2,048 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ ((128, 128, 1024),     │     6,295,552 │
│                                 │ (128, 512), (128,      │               │
│                                 │ 512), (128, 512),      │               │
│                                 │ (128, 512))            │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_1           │ (128, 128, 1024)       │         2,048 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (128, 128, 6191)       │     6,345,775 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 17,380,143 (66.30 MB)

 Trainable params: 17,380,143 (66.30 MB)

 Non-trainable params: 0 (0.00 B)

In [30]:
# Here's the code we'll use to sample for us. It has some extra steps to apply
# the temperature to the distribution, and to make sure we don't get empty
# characters in our text. Most importantly, it will keep track of our model
# state for us.

class OneStep(tf.keras.Model):
  def __init__(self, model, vectorize_layer, vocabulary, temperature=1):
    super().__init__()
    self.temperature=temperature
    self.model = model
    self.vectorize_layer = vectorize_layer
    self.vocabulary = vocabulary
    #print("initialized")

    # Create a mask to prevent "" or "[UNK]" from being generated.
    skip_ids = StringLookup(vocabulary=list(vocabulary))(['', '[UNK]'])[:, None]
    #print(skip_ids)
    #print("3")
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices = skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(vocabulary)])
    #print("4")
    self.prediction_mask = tf.sparse.to_dense(sparse_mask,validate_indices=False)
    #print("5")

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    #input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.vectorize_layer(inputs)
    #print(input_ids)

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states =  self.model(inputs=input_ids, states=states,
                                          return_state=True)
    del input_ids
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature

    # Apply the prediction mask: prevent "" or "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    del predicted_logits
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    #print(predicted_ids[0])

    # Return the characters and model state.
    return words_from_ids(predicted_ids), states


In [31]:
def produce_sample(model, vectorize_layer, vocabulary, temp, epoch, prompt):
  # Create an instance of the character generator
  #print("entered")
  one_step_model = OneStep(model, vectorize_layer, vocabulary, temp)
  #print("rand one step")
  # Now, let's generate a 1000 character chapter by giving our model "Chapter 1"
  # as its starting text
  states = None
  next_char = tf.constant([preprocess_text(prompt)])
  result = [tf.constant([prompt])]

  for n in range(200):
    next_char, states = one_step_model.generate_one_step(next_char, states=states)
    #print(next_char)
    result.append(next_char)
    #print(result)

  result = tf.strings.join(result)
  #print(result)

  # Print the results formatted.
  #print('Temp: ' + str(temp) + '\n')
  print(postprocess_text(result[0].numpy().decode('utf-8')))
  #print('\n\n')
  print('Epoch: ' + str(epoch) + '\n', file=open(path + 'tree.txt', 'a'))
  print('Temp: ' + str(temp) + '\n', file=open(path + 'tree.txt', 'a'))
  print(postprocess_text(result[0].numpy().decode('utf-8')), file=open(path + 'tree.txt', 'a'))
  print('\n\n', file=open(path + 'tree.txt', 'a'))
  del states
  del next_char
  del result

## IV. Train the model

For our purposes, we'll be using [categorical cross entropy](https://machinelearningmastery.com/cross-entropy-for-machine-learning/) as our loss function*. Also, our model will be outputting ["logits" rather than normalized probabilities](https://stackoverflow.com/questions/41455101/what-is-the-meaning-of-the-word-logits-in-tensorflow), because we'll be doing further transformations on the output later.


\* Note that since our model deals with integer encoding rather than one-hot encoding, we'll specifically be using [sparse categorical cross entropy](https://stats.stackexchange.com/questions/326065/cross-entropy-vs-sparse-cross-entropy-when-to-use-one-over-the-other).

In [32]:
# sherlock_text = getMyText()

In [33]:
if restart == False:
  model.load_weights(path + "lstm_gru_SH_modelweights_fall2023-random_urls.h5")

In [34]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
opt = tf.keras.optimizers.Adam(learning_rate=0.002)
model.compile(optimizer=opt, loss=loss)

num_epochs_total = 5
if restart:
  start_epoch = 0
else:
  start_epoch = epoch_to_pickup
for e in range(start_epoch, num_epochs_total):
  success = False
  while(success == False):
    try:
      print("epoch: ", e)
      # if e < 50:
      #   new_text = getRandomText(numbooks = 20)
      # else:
      #   new_text = sherlock_text + getRandomText(numbooks = (num_epochs_total - e)//10)
      new_text = getMyText()
      dataset = text_to_dataset(new_text)
      del new_text
      dataset = setup_dataset(dataset)
      #opt = tf.keras.optimizers.Adam(learning_rate=0.002*(0.97**e))
      #model.compile(optimizer=opt, loss=loss)
      model.optimizer.learning_rate.assign(0.002*(0.99**e))
      model.fit(dataset, epochs=1, verbose=1)
      print("finished training...")
      del dataset
      #print("saving weights...")
      #model.save_weights(path + "lstm_gru_SH_modelweights_fall2023-random_urls.h5")
      #print("weights saved...")
      for temp in [0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
        produce_sample(model,vectorize_layer,vocabulary, temp, e, 'Emma sat thinking about')
      print("samples produced...")
      gc.collect()
      print("garbage collected...")
      tf.keras.backend.clear_session()
      print("session cleared (to save memory)...")
      #tf.config.experimental.reset_all()
      success = True
    except:
      gc.collect()
      tf.keras.backend.clear_session()
      #tf.config.experimental.reset_all()
      try:
        del dataset
      except:
        print("dataset already deleted")
      print("retrying epoch: " , e)



epoch:  0
File 'gatsby.txt' found locally. Using it.
6/6 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step - loss: 7.9513
finished training...
Error during epoch 0:
Traceback (most recent call last):
  File "/tmp/ipython-input-34-784131348.py", line 42, in <cell line: 0>
    produce_sample(model, vectorize_layer, vocabulary, temp, e, 'Emma sat thinking about')
  File "/tmp/ipython-input-31-1537000287.py", line 13, in produce_sample
    next_char, states = one_step_model.generate_one_step(next_char, states=states)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/tensorflow/python/util/traceback_utils.py", line 153, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "/usr/local/lib/python3.11/dist-packages/tensorflow/python/eager/execute.py", line 59, in quick_execute
    except TypeError as e:
tensorflow.python.framework.errors_impl.InvalidArgumentError: Graph execution error:

Detected at node gatsb

KeyboardInterrupt: 